In [3]:
from sage.rings.polynomial.pbori.pbori import BooleSet
import json
import itertools
import pandas as pd
import numpy as np

We want to create a polynomial function from the binary representation of a group element to the binary representation of its inverse. To do this we construct two tables: one for the addressing function ($2\mathbb{O} \rightarrow \mathbb{Z_2^6}$), and another which simply directly inverts elements by the usual $(abc)^{-1} = c^{-1}b^{-1}a^{-1}$ formula and then stores the element and the powers used to generate it. Then, by matching the tables on the group element, we can match the parameters fed to the inverse map to our standard address.

To start we first define a Boolean ring $B$ which we will be solving over. We also define a Quaternion Algebra $Q$ and its generators, which will act as our common representation in both tables.

In [4]:
B.<x1,x2,x3,x4,x5,x6> = BooleanPolynomialRing(6)


Q_params.<z1,z2,z3,z4,z5,z6> = PolynomialRing(SR, 6)
Q.<i,j,k> = QuaternionAlgebra(Q_params, -1, -1)

u = - 1/2 * (1 + i + j + k)
t = (1 + i) / sqrt(2)

Now we define our tables. The rows of the will consist of the element, written in quaternion form, corresponding to the product $$(-1)^{p_1} u^{2 {p_2} + {p_3}} t^{p_4} j^{p_5} k^{p_6}$$ followed by the parameters $p_i$ which were used.

In [5]:
canonical_forms = pd.DataFrame(data = [{ "element" : str((-1)**p1 * j**p2 * k**p3 * u**(2*p4 + p5) * t**p6),
                                         "z1" : p1,
                                         "z2" : p2,
                                         "z3" : p3,
                                         "z4" : p4,
                                         "z5" : p5,
                                         "z6" : p6 }
                                       for p1, p2, p3, p4, p5, p6
                                       in itertools.product(range(2),
                                                            range(2),
                                                            range(2),
                                                            range(2),
                                                            range(2),
                                                            range(2))])
canonical_forms = canonical_forms[(canonical_forms.z4 + canonical_forms.z5) != 2]



inverse_table = pd.DataFrame(data = [{ "element" : str(t**(8-p6) * u**(3 - (2*p4 + p5)) * k**(4-p3) * j**(4-p2) * (-1)**(2-p1)),
                                       "x1" : p1,
                                       "x2" : p2,
                                       "x3" : p3,
                                       "x4" : p4,
                                       "x5" : p5,
                                       "x6" : p6 }
                                     for p1, p2, p3, p4, p5, p6
                                     in itertools.product(range(2),
                                                          range(2),
                                                          range(2),
                                                          range(2),
                                                          range(2),
                                                          range(2))])
inverse_table = inverse_table[(inverse_table.x4 + inverse_table.x5) != 2]

Now we join the the tables across the common element. Thus we obtain an association between an element an its inverse within the bianry representation.

In [6]:
inverse_table_idens = inverse_table.join(canonical_forms.set_index("element"),
                         on = "element")

Next up we collect, for each of the six bits of an element's address, all those sets of inverse map powers which correspond to a 0 or a 1. Then we feed this data to a specialized Sagemath solver which will produce a polynomial which upon given those inverse powers, will produce 0 or 1 exactly when the corresponding parameters were matched with a 0 or 1 respectively.

In [7]:
z1_zeros = inverse_table_idens[inverse_table_idens.z1 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z1_ones  = inverse_table_idens[inverse_table_idens.z1 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z1_zeros = set(list(map(tuple, z1_zeros)))
z1_ones  = set(list(map(tuple, z1_ones)))

z2_zeros = inverse_table_idens[inverse_table_idens.z2 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z2_ones  = inverse_table_idens[inverse_table_idens.z2 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z2_zeros = set(list(map(tuple, z2_zeros)))
z2_ones  = set(list(map(tuple, z2_ones)))

z3_zeros = inverse_table_idens[inverse_table_idens.z3 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z3_ones  = inverse_table_idens[inverse_table_idens.z3 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z3_zeros = set(list(map(tuple, z3_zeros)))
z3_ones  = set(list(map(tuple, z3_ones)))

z4_zeros = inverse_table_idens[inverse_table_idens.z4 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z4_ones  = inverse_table_idens[inverse_table_idens.z4 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z4_zeros = set(list(map(tuple, z4_zeros)))
z4_ones  = set(list(map(tuple, z4_ones)))

z5_zeros = inverse_table_idens[inverse_table_idens.z5 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z5_ones  = inverse_table_idens[inverse_table_idens.z5 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z5_zeros = set(list(map(tuple, z5_zeros)))
z5_ones  = set(list(map(tuple, z5_ones)))

z6_zeros = inverse_table_idens[inverse_table_idens.z6 == 0].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z6_ones  = inverse_table_idens[inverse_table_idens.z6 == 1].loc[:,["x1","x2","x3","x4","x5","x6"]].values
z6_zeros = set(list(map(tuple, z6_zeros)))
z6_ones  = set(list(map(tuple, z6_ones)))

In [8]:
z1p = B.interpolation_polynomial(z1_zeros,z1_ones)
z2p = B.interpolation_polynomial(z2_zeros,z2_ones)
z3p = B.interpolation_polynomial(z3_zeros,z3_ones)
z4p = B.interpolation_polynomial(z4_zeros,z4_ones)
z5p = B.interpolation_polynomial(z5_zeros,z5_ones)
z6p = B.interpolation_polynomial(z6_zeros,z6_ones)

Then we do the final test. Theres a little bit of trickery required to make the types match. Its unimportant, but we have to cast to a less algorithically specialized ring. After casting all the polynomials, we run through each parameter arrangement and generate the corresponding element. We simulatenously run each of our 6 polynomials on that set of parameters fed to the inverse function and generate the corresponding element. We then compare to ensure we get the same result. If everything works, you should see "True" after running this!

In [9]:
B_as_Z =  GF(2)["x1","x2","x3","x4","x5","x6"]
z1_poly = B_as_Z(B.interpolation_polynomial(z1_zeros,z1_ones))
z2_poly = B_as_Z(B.interpolation_polynomial(z2_zeros,z2_ones))
z3_poly = B_as_Z(B.interpolation_polynomial(z3_zeros,z3_ones))
z4_poly = B_as_Z(B.interpolation_polynomial(z4_zeros,z4_ones))
z5_poly = B_as_Z(B.interpolation_polynomial(z5_zeros,z5_ones))
z6_poly = B_as_Z(B.interpolation_polynomial(z6_zeros,z6_ones))



def powers_to_el(arg1,arg2,arg3,arg4,arg5,arg6):
    return ((-1)^arg1) * (j^arg2) * (k^arg3) * (u^((2 * arg4) + arg5))  * (t^arg6)

def invert_el(arg1,arg2,arg3,arg4,arg5,arg6):
    return t**(8-arg6) * u**(3 - (2*arg4 + arg5)) * k**(4-arg3) * j**(4-arg2) * (-1)**(2 - arg1)


results = []
for p1, p2, p3, p4, p5, p6 in list(itertools.product(range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2))):


    #Ignore impossible case (where our polys fail!)
    if (p4 and p5):
        continue

    z1_pow = int(z1_poly(p1,p2,p3,p4,p5,p6))
    z2_pow = int(z2_poly(p1,p2,p3,p4,p5,p6))
    z3_pow = int(z3_poly(p1,p2,p3,p4,p5,p6))
    z4_pow = int(z4_poly(p1,p2,p3,p4,p5,p6))
    z5_pow = int(z5_poly(p1,p2,p3,p4,p5,p6))
    z6_pow = int(z6_poly(p1,p2,p3,p4,p5,p6))

    inverse = invert_el(p1,p2,p3,p4,p5,p6)
    potential_result = powers_to_el(z1_pow,z2_pow,z3_pow,z4_pow,z5_pow,z6_pow)

    results.append(inverse == potential_result)

print(all(results))

True


In [13]:
z1p

x1 + x2*x3*x6 + x2*x3 + x2*x5*x6 + x2*x6 + x2 + x3*x4*x6 + x3*x6 + x3 + x4*x6 + x6

In [16]:
z3p(x6=1)

x2*x4 + x2 + x3*x4 + x3*x5 + x4 + x5 + 1

In [12]:
z2p - z3p

x2*x5 + x2 + x3*x4 + x3 + x4*x6

In [11]:
z4p(x6=1)

x4

In [12]:
z5p(x6=1)

x5

In [13]:
z6p

x6

In [14]:
z2p(x6=1) - z3p(x6=1)

x2*x5 + x2 + x3*x4 + x3 + x4

In [15]:
z1_poly

x2*x3*x6 + x3*x4*x6 + x2*x5*x6 + x2*x3 + x2*x6 + x3*x6 + x4*x6 + x1 + x2 + x3 + x6

In [16]:
z2_poly

x3*x4*x6 + x2*x5*x6 + x2*x4 + x3*x4 + x3*x5 + x2*x6 + x3*x6 + x5*x6 + x2 + x6

In [17]:
z5p(x6=0)

x4